In [2]:
import os
from ultralytics import YOLO
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
import glob
import numpy as np
import shutil

In [3]:
def show_bbox(image_path):
    # convert image path to label path
    label_path = image_path.replace('/images/', '/darknet/')
    label_path = label_path.replace('.jpg', '.txt')

    # Open the image and create ImageDraw object for drawing
    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)

    with open(label_path, 'r') as f:
        for line in f.readlines():
            # Split the line into five values
            label, x, y, w, h = line.split(' ')

            # Convert string into float
            x = float(x)
            y = float(y)
            w = float(w)
            h = float(h)

            # Convert center position, width, height into
            # top-left and bottom-right coordinates
            W, H = image.size
            x1 = (x - w/2) * W
            y1 = (y - h/2) * H
            x2 = (x + w/2) * W
            y2 = (y + h/2) * H

            # Draw the bounding box with red lines
            draw.rectangle((x1, y1, x2, y2),
                           outline=(255, 0, 0), # Red in RGB
                           width=5)             # Line width
    image.show()


def get_filenames(folder):
    filenames = set()
    
    for path in glob.glob(os.path.join(folder, '*.jpg')):
        # Extract the filename
        filename = os.path.split(path)[-1]        
        filenames.add(filename)

    return filenames


In [4]:
# Dog and cat image filename sets
dog_images = get_filenames('download/dog/images')
cat_images = get_filenames('download/cat/images')

In [ ]:
# # Create a folder structure for YOLOv5 training
# if not os.path.exists('data'):
#     for folder in ['images', 'labels']:
#         for split in ['train', 'val', 'test']:
#             os.makedirs(f'data/{folder}/{split}')

In [ ]:
dog_images = np.array(list(dog_images))
cat_images = np.array(list(cat_images))

# Use the same random seed for reproducability
np.random.seed(42)
np.random.shuffle(dog_images)
np.random.shuffle(cat_images)


def split_dataset(animal, image_names, train_size, val_size):
    for i, image_name in enumerate(image_names):
        # Label filename
        label_name = image_name.replace('.jpg', '.txt')
        
        # Split into train, val, or test
        if i < train_size:
            split = 'train'
        elif i < train_size + val_size:
            split = 'val'
        else:
            split = 'test'
        
        # Source paths
        source_image_path = f'download/{animal}/images/{image_name}'
        source_label_path = f'download/{animal}/darknet/{label_name}'

        # Destination paths
        target_image_folder = f'data/images/{split}'
        target_label_folder = f'data/labels/{split}'

        # Copy files
        shutil.copy(source_image_path, target_image_folder)
        shutil.copy(source_label_path, target_label_folder)

# # Cat data
# split_dataset('cat', cat_images, train_size=400, val_size=50)

# # Dog data (reduce the number by 1 for each set due to three duplicates)
# split_dataset('dog', dog_images, train_size=399, val_size=49)

In [7]:
model = YOLO("yolo11n.pt")
results = model.train(data="cats_and_dogs.yaml", epochs=100, imgsz=640)

New https://pypi.org/project/ultralytics/8.3.160 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.159 🚀 Python-3.10.12 torch-2.7.1+cu126 CUDA:0 (Quadro RTX 6000, 24024MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=cats_and_dogs.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train5, nbs=64, nms=False, opset=None, optimize=False

train: Scanning /home/decla_5ay7wb/RIPS25-AnalogDevices-ObjectDetection/data/labels/train... 796 images, 0 backgrounds, 0 corrupt: 100%|██████████| 796/796 [00:01<00:00, 744.69it/s]


train: New cache created: /home/decla_5ay7wb/RIPS25-AnalogDevices-ObjectDetection/data/labels/train.cache
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1502.1±1505.4 MB/s, size: 390.1 KB)


val: Scanning /home/decla_5ay7wb/RIPS25-AnalogDevices-ObjectDetection/data/labels/val... 99 images, 0 backgrounds, 0 corrupt: 100%|██████████| 99/99 [00:00<00:00, 815.59it/s]

val: New cache created: /home/decla_5ay7wb/RIPS25-AnalogDevices-ObjectDetection/data/labels/val.cache


...

    100/100       4.2G      0.439     0.3382     0.9195         14        640: 100%|██████████| 50/50 [00:05<00:00,  9.28it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:00<00:00,  9.85it/s]

                   all         99        114       0.81      0.788      0.822       0.63



100 epochs completed in 0.176 hours.
Optimizer stripped from /home/decla_5ay7wb/RIPS25-AnalogDevices-ObjectDetection/runs/detect/train5/weights/last.pt, 5.5MB
Optimizer stripped from /home/decla_5ay7wb/RIPS25-AnalogDevices-ObjectDetection/runs/detect/train5/weights/best.pt, 5.5MB

Validating /home/decla_5ay7wb/RIPS25-AnalogDevices-ObjectDetection/runs/detect/train5/weights/best.pt...
Ultralytics 8.3.159 🚀 Python-3.10.12 torch-2.7.1+cu126 CUDA:0 (Quadro RTX 6000, 24024MiB)
YOLO11n summary (fused): 100 layers, 2,582,542 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:00<00:00,  5.13it/s]


                   all         99        114      0.834       0.75       0.85      0.648
                   cat         50         54      0.805       0.87      0.912      0.736
                   dog         49         60      0.863       0.63      0.788      0.561
Speed: 0.1ms preprocess, 0.8ms inference, 0.0ms loss, 2.6ms postprocess per image
Results saved to /home/decla_5ay7wb/RIPS25-AnalogDevices-ObjectDetection/runs/detect/train5
